In [1]:
import numpy as np
import pandas as pd
from mido import MidiFile
from collections import namedtuple

In [2]:
midi_dir = "../midi/"
TimedNote = namedtuple('TimedNote', ['note', 'start_tick', 'end_tick'])
CustomMessage = namedtuple('CustomMessage', ['start_tick', 'note', 'on'])

In [3]:
"""
Structure for keeping track of all notes playing at the same time
"""
class ActiveNotes:
    def __init__(self):
        self.cur_tick = 0
        self.currently_playing = np.zeros([128], dtype='bool')
        # Array to prevent adding the same highest note twice
        # (f.e. some note gets turned off and some note which has been added to the melody already
        # might still be playing and gets evaluated as highest note again)
        self.already_in_melody = np.zeros([128], dtype='bool')

    def note_on(self, note):
        self.currently_playing[note] = True

    def note_off(self, note):
        self.currently_playing[note] = False
        self.already_in_melody[note] = False

    """
    Return highest note and ticks when it started/stopped
    delta_tick: time since latest MIDI Message
    """
    def get_highest_note(self, delta_tick):
        previous_tick = self.cur_tick
        self.cur_tick = self.cur_tick + delta_tick
        for i in range(127, -1, -1):
            # Found the highest note that's currently playing
            if self.currently_playing[i]:
                # It hasn't been added to melody yet; add it
                if not self.already_in_melody[i]:
                    self.already_in_melody[i] = True
                    return TimedNote(i, previous_tick, self.cur_tick)

                # Has been added to melody; stop the search for highest note
                else:
                    break

        return TimedNote(-1, previous_tick, self.cur_tick)


In [4]:
def get_highest_melody(file, tracks_to_merge, tick_ignore):
    mid = MidiFile(file)
    highest_melodies = []
    merged_melody = []

    # Process all tracks separately first
    for track_no in tracks_to_merge:
        an = ActiveNotes()
        highest_melody = []
        additive_tick = 0
        for msg in mid.tracks[track_no]:
            if msg.type == 'note_on':
                # Not instant; some time has passed => evaluate previous highest note
                tick = msg.time + additive_tick
                additive_tick = 0
                if tick != 0:
                    res = an.get_highest_note(tick)
                    # Filter out cases when nothing is playing
                    if res.note != -1:
                        # highest_melody.append(res)
                        # Add turn on/turn off messages
                        highest_melody.append(CustomMessage(res.start_tick, res.note, 1))
                        highest_melody.append(CustomMessage(res.end_tick, res.note, 0))

                if msg.velocity != 0:
                    an.note_on(msg.note)
                else:
                    an.note_off(msg.note)

            # Some messages just change time for no FUCKING REASON
            else:
                additive_tick = additive_tick + msg.time

            # if msg.type in ['set_tempo', 'control_change', 'time_signature']:
            #    additive_tick = additive_tick + msg.time

        highest_melodies.append(highest_melody)

    # Merge all highest melody messages into one list
    mega_list = []
    for i in highest_melodies:
        mega_list = mega_list + i

    # Sort all messages by starting tick
    mega_list.sort(key=lambda custom_msg: custom_msg.start_tick)
    an = ActiveNotes()
    cur_tick = mega_list[0][0]

    for custom_msg in mega_list:
        # Treat a very small time difference between messages as if the events happened simultaneously
        # So that the higher note gets judged correctly
        if abs(custom_msg.start_tick - cur_tick) > tick_ignore:
            res = an.get_highest_note(0)
            if res.note != -1:
                merged_melody.append(res.note)

        if custom_msg.on:
            an.note_on(custom_msg.note)

        else:
            an.note_off(custom_msg.note)

        cur_tick = custom_msg.start_tick


    return merged_melody, mega_list


In [5]:
res = get_highest_melody("../backend/midi/bach_846.mid", [1, 2], 10)
print(res[0])

[60, 64, 67, 72, 76, 67, 72, 76, 60, 64, 67, 72, 76, 67, 72, 76, 60, 62, 69, 74, 77, 69, 74, 77, 60, 62, 69, 74, 77, 69, 74, 77, 59, 62, 67, 74, 77, 67, 74, 77, 59, 62, 67, 74, 77, 67, 74, 77, 60, 64, 67, 72, 76, 67, 72, 76, 60, 64, 67, 72, 76, 67, 72, 76, 60, 64, 69, 76, 81, 69, 76, 81, 60, 64, 69, 76, 81, 69, 76, 81, 60, 62, 66, 69, 74, 66, 69, 74, 60, 62, 66, 69, 74, 66, 69, 74, 59, 62, 67, 74, 79, 67, 74, 79, 59, 62, 67, 74, 79, 67, 74, 79, 59, 60, 64, 67, 72, 64, 67, 72, 59, 60, 64, 67, 72, 64, 67, 72, 57, 60, 64, 67, 72, 64, 67, 72, 57, 60, 64, 67, 72, 64, 67, 72, 50, 57, 62, 66, 72, 62, 66, 72, 50, 57, 62, 66, 72, 62, 66, 72, 55, 59, 62, 67, 71, 62, 67, 71, 55, 59, 62, 67, 71, 62, 67, 71, 55, 58, 64, 67, 73, 64, 67, 73, 55, 58, 64, 67, 73, 64, 67, 73, 53, 57, 62, 69, 74, 62, 69, 74, 53, 57, 62, 69, 74, 62, 69, 74, 53, 56, 62, 65, 71, 62, 65, 71, 53, 56, 62, 65, 71, 62, 65, 71, 52, 55, 60, 67, 72, 60, 67, 72, 52, 55, 60, 67, 72, 60, 67, 72, 52, 53, 57, 60, 65, 57, 60, 65, 52, 53,

In [36]:
# Load a midi file
midd = MidiFile("../backend/midi/piano_midi_de/appass_1.mid")
# Print notes in midi track
for track in midd.tracks:
    print("trck: " + track.name)

print(midd.type)

trck: Sonate Nr. 23 Op. 57
trck: Piano right
trck: Piano left
trck: Pedal
trck: Beethoven: Sonate Op. 57 (Appassionata) 1. Satz
trck: Copyright © 2001 by Bernd Krüger
trck: http://www.piano-midi.de
trck: Edition: 2012-07-21
trck: Spur 8
trck: Spur 9
trck: Spur 10
1


In [37]:
for msgg in midd.tracks[0]:
    # Not instant; some time has passed => evaluate previous highest note
    print(msgg)

MetaMessage('track_name', name='Sonate Nr. 23 Op. 57', time=0)
MetaMessage('track_name', name='Appassionata', time=0)
MetaMessage('copyright', text='Copyright © 2001 by Bernd Krueger', time=0)
MetaMessage('text', text='Ludwig van Beethoven', time=0)
MetaMessage('text', text='Allegro assai', time=0)
MetaMessage('text', text='Fertiggestellt am 26.9.2001\n', time=0)
MetaMessage('text', text='Update am 23.10.01\n', time=0)
MetaMessage('text', text='Normierung: 23.12.2002\n', time=0)
MetaMessage('text', text='Update am 21.7.2012\n', time=0)
MetaMessage('text', text='Dauer: 9:22 Minuten\n', time=0)
MetaMessage('smpte_offset', frame_rate=25, hours=32, minutes=0, seconds=3, frames=0, sub_frames=0, time=0)
MetaMessage('time_signature', numerator=12, denominator=8, clocks_per_click=12, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='Ab', time=0)
MetaMessage('set_tempo', tempo=361816, time=0)
MetaMessage('set_tempo', tempo=383779, time=2160)
MetaMessage('set_tempo', tempo

In [34]:
def get_metadata(file):
    # Load a midi file
    mid = MidiFile(file)
    track_name = ""
    author = ""
    # First track contains data
    for msg in mid.tracks[0]:
        # Not instant; some time has passed => evaluate previous highest note
        if msg.type == "track_name":
            if track_name == "":
                track_name = msg.name
            else:
                track_name = track_name + ": " + msg.name

        # First text message contains the author
        if msg.type == "text":
            author = msg.text
            break

    # Second and third tracks contain right/left hand
    melody = get_highest_melody(file, [1, 2], 5)[0]
    return {
        "track_name": track_name,
        "author": author,
        "melody": melody
    }

In [35]:
res = get_metadata("../backend/midi/chpn_op53.mid")
print(res)

{'track_name': 'Chopin Polonaise in Ab major Opus 53', 'author': 'Frederic Chopin', 'melody': [63, 57, 58, 59, 60, 61, 62, 63, 58, 68, 67, 59, 58, 52, 51, 59, 58, 52, 51, 59, 58, 52, 51, 59, 58, 52, 51, 59, 58, 52, 51, 64, 58, 59, 60, 61, 62, 63, 64, 59, 73, 71, 61, 59, 54, 52, 61, 59, 54, 52, 61, 59, 54, 52, 61, 59, 54, 52, 61, 59, 54, 52, 65, 62, 63, 64, 65, 67, 68, 69, 65, 72, 70, 64, 65, 66, 67, 68, 69, 70, 65, 77, 75, 77, 68, 67, 75, 77, 68, 67, 75, 77, 67, 63, 75, 77, 67, 63, 75, 77, 70, 67, 75, 77, 70, 63, 75, 77, 70, 63, 75, 77, 70, 63, 75, 77, 70, 63, 75, 77, 70, 63, 75, 77, 70, 63, 75, 77, 75, 63, 75, 56, 72, 73, 77, 75, 77, 75, 74, 75, 56, 72, 73, 77, 75, 77, 75, 74, 77, 73, 75, 77, 75, 74, 75, 63, 70, 78, 77, 72, 73, 66, 54, 66, 65, 61, 66, 78, 77, 65, 77, 58, 73, 75, 78, 77, 78, 77, 76, 77, 58, 79, 80, 84, 79, 84, 82, 82, 75, 82, 80, 80, 75, 80, 79, 79, 74, 79, 77, 77, 70, 77, 75, 73, 68, 73, 56, 72, 65, 67, 68, 67, 68, 65, 67, 68, 70, 68, 70, 67, 68, 69, 70, 72, 73, 72, 7